# Strands agent with an MCP

In [1]:
from mcp import ClientSession, stdio_client, StdioServerParameters
from strands import Agent
from strands.models.ollama import OllamaModel
from strands.tools.mcp import MCPClient

## Strans only supports tools, not prompts or resources

Let's get the database schema from the MCP server the old fashioned way and add it to the system prompt.

Uses [mcp-experiemnt](https://github.com/lewiesnyder/mcp-experiment) to build a simple sql agent

In [2]:
# 1. Get therethereschema from MCP
stdio_params = StdioServerParameters(command="docker", args=["run", "-i", "--rm", "mcp-experiment"])
async with stdio_client(stdio_params) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        await session.initialize()
        meta, schema = await session.read_resource("schema://main")

system_prompt = f"""
You are a helpful assistant that has access to various tools. Answer the user's queries to the best of your ability.

You have access to the following schema for a database of music artists:
{schema[1][0].text}

## Requirements
- THINK before you act.
- OBSERVE after you act.
- THINK again before you act again.
"""

## Using llama3.2 hosted in ollama locally

You could use the [litellm provider](https://strandsagents.com/latest/user-guide/concepts/model-providers/litellm/) and use the llm of your choice:
```python
import os
from dotenv import load_dotenv
from strands.models.litellm import LiteLLMModel


load_dotenv()

# ollama
model = LiteLLMModel(
    model_id="ollama/llama3.2"
)

# claude
model = LiteLLMModel(
    client_args={
        "api_key": os.get('ANTHROPIC_API_KEY'),
    },
    # **model_config
    model_id="anthropic/claude-3-7-sonnet-20250219",
    params={
        "max_tokens": 1000,
        "temperature": 0.7,
    }
)
```

In [3]:
model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.2"
)

## Create the MCP client

In [4]:
mcp_client = MCPClient(lambda: stdio_client(stdio_params))

## Invoke the agent

In [5]:
with mcp_client:
    tools = mcp_client.list_tools_sync()
    agent = Agent(system_prompt=system_prompt, model=model, tools=tools)

    agent("What are the top three tracks by sales?")



Tool #1: query_data
It seems that the SQL query is not correctly formatted. SQLite does not support the `TOP` keyword, which is typically used in Microsoft SQL Server.

Here's a revised version of the query that uses the `ORDER BY` clause to retrieve the top 3 tracks by sales:


Tool #2: query_data

Tool #3: query_data
The top three tracks by sales are:

1. "The Trooper" with a total of 4.95
2. "Walkabout" with a total of 3.98
3. "The Woman King" with a total of 3.98